<a href="https://colab.research.google.com/github/alteregoyishan/CASA0005_homework4/blob/main/W05_quiz_2025.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

# Week 5 Quiz

This notebook contains the SQL Quiz for Week 5. Section 1 uses the New York City data we know and love. Section 2 will conduct a spatial damage assessment across the Gaza strip.

### INSTRUCTIONS:

Run this notebook in Google Colab. The answer to each quesiton will be a number or a string. Input these into the corresponding question on Moodle. You have 90 minutes to attempt the quiz, so if you get stuck on a question, move on.

Make sure you run all of the cells of code in order. If you run into serious problems, try clicking on the "runtime" tab above and selecting "restart session and run all".

# Section 1



This URL: `https://s3.amazonaws.com/s3.cleverelephant.ca/postgis-workshop-2020.zip` points to a .zip file containing shapefiles associated with the NYC data that we've been using in class so far. You can choose how to use sql, either the `con.sql()` syntax or the `%%sql` syntax. Either way, use the code cell below to install and import the necessary packages.

In [38]:
 %pip install duckdb leafmap

In [39]:
import duckdb

%load_ext sql

The sql extension is already loaded. To reload it, use:
  %reload_ext sql


In [43]:
url = "https://s3.amazonaws.com/s3.cleverelephant.ca/postgis-workshop-2020.zip"
leafmap.download_file(url, unzip=True, overwrite=True)

Downloading...
From: https://s3.amazonaws.com/s3.cleverelephant.ca/postgis-workshop-2020.zip
To: /content/postgis-workshop-2020.zip
100%|██████████| 22.5M/22.5M [00:00<00:00, 58.0MB/s]


Extracting files...


'/content/postgis-workshop-2020.zip'

In [44]:
con = duckdb.connect('nyc_data.db')

In [45]:
con.install_extension('spatial')
con.load_extension('spatial')

In [46]:
con.sql("SHOW TABLES;")

┌─────────┐
│  name   │
│ varchar │
├─────────┤
│ 0 rows  │
└─────────┘

## Question 1

Firslty, please provide the link to your colab notebook in the Moodle quiz using the following steps:

1. In the top right corner, click the "share" button
2. In the popup, click on "restricted" and change this to "anyone with the link"
3. Finally, click "copy link", and paste the link into the box for Question 1 on moodle.

Once you've done this, download and unzip the data, then create the following tables using the corresponding shapefiles.

- nyc_neighborhoods
- nyc_census_blocks
- nyc_homicides
- nyc_streets
- nyc_subway_stations

In [52]:
import geopandas as gpd

shapefile_path = "/content/postgis-workshop/data/nyc_neighborhoods.shp"


con.sql(f"""
    CREATE TABLE nyc_neighborhoods AS
    SELECT * FROM ST_Read('{shapefile_path}');
""")



✅ `nyc_neighborhoods` 表已成功创建！


In [53]:
shapefile_path = "/content/postgis-workshop/data/nyc_census_blocks.shp"

con.sql(f"""
    CREATE TABLE nyc_census_blocks AS
    SELECT * FROM ST_Read('{shapefile_path}');
""")

In [54]:
shapefile_path = "/content/postgis-workshop/data/nyc_streets.shp"

con.sql(f"""
    CREATE TABLE nyc_streets AS
    SELECT * FROM ST_Read('{shapefile_path}');
""")

In [56]:
shapefile_path = "/content/postgis-workshop/data/nyc_homicides.shp"

con.sql(f"""
    CREATE TABLE nyc_homicides AS
    SELECT * FROM ST_Read('{shapefile_path}');
""")

In [60]:
shapefile_path = "/content/postgis-workshop/data/nyc_subway_stations.shp"

con.sql(f"""
    CREATE TABLE nyc_subway_stations AS
    SELECT * FROM ST_Read('{shapefile_path}');
""")

## Question 2:
What is the longest street in New York? Note, it may be split up into multiple segments! Ignore missing and null values.


In [55]:
con.sql("""SELECT name, SUM(ST_Length(geom)) AS total_length
FROM nyc_streets
WHERE name IS NOT NULL
GROUP BY name
ORDER BY total_length DESC
LIMIT 1;
""")

┌───────────────────┬────────────────────┐
│       NAME        │    total_length    │
│      varchar      │       double       │
├───────────────────┼────────────────────┤
│ Grand Central Pky │ 40210.484038252165 │
└───────────────────┴────────────────────┘

## Question 3:

Which borough had the fewest daytime shootings in 2009?

In [59]:
con.sql("DESCRIBE nyc_homicides").show()

┌─────────────┬─────────────┬─────────┬─────────┬─────────┬─────────┐
│ column_name │ column_type │  null   │   key   │ default │  extra  │
│   varchar   │   varchar   │ varchar │ varchar │ varchar │ varchar │
├─────────────┼─────────────┼─────────┼─────────┼─────────┼─────────┤
│ INCIDENT_D  │ DATE        │ YES     │ NULL    │ NULL    │ NULL    │
│ BORONAME    │ VARCHAR     │ YES     │ NULL    │ NULL    │ NULL    │
│ NUM_VICTIM  │ VARCHAR     │ YES     │ NULL    │ NULL    │ NULL    │
│ PRIMARY_MO  │ VARCHAR     │ YES     │ NULL    │ NULL    │ NULL    │
│ ID          │ BIGINT      │ YES     │ NULL    │ NULL    │ NULL    │
│ WEAPON      │ VARCHAR     │ YES     │ NULL    │ NULL    │ NULL    │
│ LIGHT_DARK  │ VARCHAR     │ YES     │ NULL    │ NULL    │ NULL    │
│ YEAR        │ BIGINT      │ YES     │ NULL    │ NULL    │ NULL    │
│ geom        │ GEOMETRY    │ YES     │ NULL    │ NULL    │ NULL    │
└─────────────┴─────────────┴─────────┴─────────┴─────────┴─────────┘



In [66]:
con.sql(" select *  FROM nyc_homicides").df()

,INCIDENT_D,BORONAME,NUM_VICTIM,PRIMARY_MO,ID,WEAPON,LIGHT_DARK,YEAR,geom
0,2008-01-01,Brooklyn,1,None,7,gun,D,2008,"[0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 1, 0, 0, ..."
1,2008-01-04,Manhattan,1,None,14,gun,D,2008,"[0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 1, 0, 0, ..."
2,2008-01-05,Queens,1,None,15,gun,D,2008,"[0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 1, 0, 0, ..."
3,2008-01-04,Queens,1,None,16,knife,D,2008,"[0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 1, 0, 0, ..."
4,2008-01-05,Queens,1,None,18,gun,D,2008,"[0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 1, 0, 0, ..."
...,...,...,...,...,...,...,...,...,...
3979,2010-10-11,The Bronx,1,None,4269,gun,None,2010,"[0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 1, 0, 0, ..."
3980,2010-10-06,The Bronx,1,None,4271,knife,None,2010,"[0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 1, 0, 0, ..."
3981,2011-07-26,The Bronx,1,None,4282,gun,None,2011,"[0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 1, 0, 0, ..."
3982,2011-07-28,The Bronx,1,None,4284,gun,None,2011,"[0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 1, 0, 0, ..."


In [69]:
con.sql("""SELECT BORONAME, COUNT(*) AS shooting_count
FROM nyc_homicides
WHERE YEAR = 2009
AND WEAPON = 'gun'  -- 如果枪击案的武器是 firearm
AND LIGHT_DARK = 'D'
GROUP BY BORONAME
ORDER BY shooting_count ASC
LIMIT 1;
""")

┌───────────────┬────────────────┐
│   BORONAME    │ shooting_count │
│    varchar    │     int64      │
├───────────────┼────────────────┤
│ Staten Island │              6 │
└───────────────┴────────────────┘

## Question 4:

What is the total population of the census blocks served by the L train?

In [70]:
con.sql("""SELECT SUM(cb.popn_total) AS total_population
FROM nyc_census_blocks cb
JOIN nyc_subway_stations ss
ON ST_DWithin(ss.geom, cb.geom, 800)
WHERE ss.routes LIKE '%L%';
""")


┌──────────────────┐
│ total_population │
│      int128      │
├──────────────────┤
│          1145236 │
└──────────────────┘

## Question 5

which subway station had the largest number of homicides within 1km?

In [71]:
con.sql("""
SELECT ss.name AS subway_station,
       COUNT(h.geom) AS homicide_count
FROM nyc_subway_stations ss
JOIN nyc_homicides h
ON ST_DWithin(ss.geom, h.geom, 1000)
GROUP BY ss.name
ORDER BY homicide_count DESC
LIMIT 1;
""")

┌────────────────┬────────────────┐
│ subway_station │ homicide_count │
│    varchar     │     int64      │
├────────────────┼────────────────┤
│ 125th St       │            305 │
└────────────────┴────────────────┘

# Section 2

This section explores building damage in the Gaza Strip resulting from the ongoing war. You will conduct a geospatial analysis of building damage using two datasets:

1. [Humantiarian Open Street Map](https://www.hotosm.org/projects/gaza-building-footprints-pre-conflict-update-2024/) building footprints for the Gaza Strip
  * data: https://storage.googleapis.com/qm2/Gaza_Buildings_2.geojson.zip
2. [Damage points](https://unosat.org/products/3984) from the UN Satellite Agency (UNOSAT).
  * data: https://storage.googleapis.com/qm2/UNOSAT_GAZA_20240503_2.zip


The Coordinate Reference System of these datasets is EPSG:4326; therefore, stock functions like ST_DISTANCE() will yield values in degrees, not meters. Be mindful of this in your analysis, and use the DuckDB spatial functions [documentation](https://duckdb.org/docs/extensions/spatial/functions#st_geomfromtext) to your advantage.


First, download and unzip the data, and create two tables: `gaza_buildings`, containing the building footprint data, and `damage_points` containing the UNOSAT damage points. Make sure to set a spatial index on these tables, this will make your queries run much faster!

In [72]:
buildings_url = "https://storage.googleapis.com/qm2/Gaza_Buildings_2.geojson.zip"
damage_url = "https://storage.googleapis.com/qm2/UNOSAT_GAZA_20240503_2.zip"

leafmap.download_file(buildings_url, unzip=True, overwrite=True)
leafmap.download_file(damage_url, unzip=True, overwrite=True)


Downloading...
From: https://storage.googleapis.com/qm2/Gaza_Buildings_2.geojson.zip
To: /content/Gaza_Buildings_2.geojson.zip
100%|██████████| 15.0M/15.0M [00:00<00:00, 238MB/s]


Extracting files...


Downloading...
From: https://storage.googleapis.com/qm2/UNOSAT_GAZA_20240503_2.zip
To: /content/UNOSAT_GAZA_20240503_2.zip
100%|██████████| 4.98M/4.98M [00:00<00:00, 159MB/s]


Extracting files...


'/content/UNOSAT_GAZA_20240503_2.zip'

In [75]:
con.sql("""CREATE TABLE gaza_buildings AS
SELECT *
FROM ST_Read('Gaza_Buildings_2.geojson');

CREATE INDEX gaza_buildings_geometry_idx
ON gaza_buildings
USING RTREE(geom);""")

FloatProgress(value=0.0, layout=Layout(width='auto'), style=ProgressStyle(bar_color='black'))

In [86]:
con.sql("""DROP TABLE IF EXISTS damage_points;""")

In [87]:
con.sql("""CREATE TABLE damage_points AS
SELECT * FROM ST_Read('UNOSAT_GAZA_20240503_2.shp');
""")

FloatProgress(value=0.0, layout=Layout(width='auto'), style=ProgressStyle(bar_color='black'))

## Question 6

Create a table called "gaza_buildings_damaged" which is comprised only of building footprints that intersect with a UNOSAT damage point. How many damaged buildings are there in Gaza?

In [89]:
con.sql("""CREATE TABLE gaza_buildings_damaged AS
SELECT gb.*
FROM gaza_buildings gb
JOIN damage_points dp
ON ST_Intersects(gb.geom, dp.geom);
""")

FloatProgress(value=0.0, layout=Layout(width='auto'), style=ProgressStyle(bar_color='black'))

In [90]:
con.sql("""SELECT COUNT(*) AS damaged_building_count FROM gaza_buildings_damaged;
""")

┌────────────────────────┐
│ damaged_building_count │
│         int64          │
├────────────────────────┤
│                 109495 │
└────────────────────────┘

## Question 7

What is the total area (in square kilometers) of damaged buildings in Gaza? note: there are TWO area functions in duckDB, and only one of them returns answers in meters.

In [94]:
con.sql("""SELECT SUM(ST_Area(ST_Transform(geom, 'EPSG:3857'))) / 1e6 AS total_area_km2
FROM gaza_buildings_damaged;
""")

BinderException: Binder Error: No function matches the given name and argument types 'ST_Transform(GEOMETRY, STRING_LITERAL)'. You might need to add explicit type casts.
	Candidate functions:
	ST_Transform(BOX_2D, VARCHAR, VARCHAR) -> BOX_2D
	ST_Transform(BOX_2D, VARCHAR, VARCHAR, BOOLEAN) -> BOX_2D
	ST_Transform(POINT_2D, VARCHAR, VARCHAR) -> POINT_2D
	ST_Transform(POINT_2D, VARCHAR, VARCHAR, BOOLEAN) -> POINT_2D
	ST_Transform(GEOMETRY, VARCHAR, VARCHAR) -> GEOMETRY
	ST_Transform(GEOMETRY, VARCHAR, VARCHAR, BOOLEAN) -> GEOMETRY


## Question 8

What percentage of hospitals in Gaza have been damaged?


## Question 9

The UNOSAT data contains a column called "Governorat" (shapefiles only allow column names of up to 10 characters). Calculate the number of damaged schools in each governorate.

## Question 10

The "Dar Al-Shifa Hospital" was the largest medical complex in the gaza strip. Find the hospital's building footprint, and calculate the proportion of damaged buildings within 1 kilometer.